In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
from datetime import timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy, es usada para concatenar al nombre del indice principal previa inserción
* ahora, es la fecha en un formato requerido para ES

In [3]:
now = datetime.now()
format_ES = "%Y.%m.%d"
fecha_hoy = str(now.strftime(format_ES))
ahora_format = "%Y-%m-%d"'T'"%H:%M:%S"
ahora = str(now.strftime(ahora_format))

### Definiendo indice principal con fecha de hoy

In [4]:
indice = parametros.gestion_tablero14_index 
indice_control = parametros.tableros_mintic_control

### Función para generar JSON compatible con ES

In [5]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [6]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
           "_source": ["tablero14.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero14"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['tablero14.fechaControl']
except Exception as e:
    print(e)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-14T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)



ultima fecha para control de ejecucion: 2021-07-01T00:00:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede.)

In [7]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud', 'longitud','id_Beneficiario','COD_ISO','codDanesede',
                           'cod_servicio','codDaneMuni','nombre_centro_pob','codCentroPoblado','codDaneInstitucionEdu',
                           'tipoSitio','detalleSitio','energia','region','matricula','DDA','grupoDesc','estadoInstalacion',
                           'nombreInstitucionEd']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except Exception as e:
    print(e)
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()

In [8]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.tablero14location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.tablero14location']=datos_semilla['gestion.tablero14location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)


### Leyendo indice servicemanager-interacciones

Se lee la información de los De Service Manager. En la lectura se traen todas las interacciones reportadas por servicemanager, con la información de la misma.

* id_beneficiario, es la llave para cruzar con cada centro de conexión.
* variable1 y variable2, son la tipo y subtipo de la interaccion.
* current_phase, estado del ticket
* clr_txt_assignment, es el sujeto asignado al ticket
* owner_name, usuario del ticket
* open_time y close_time, son las fecha de apertura y cierre del ticket
* resolution, respuesta al ticket
* incident_id, numero del ticket
* source, fuente del ticket(email o telefono)
* severity, gravedad del ticket
* clr_txt_codcategorymtc categoria segun mintic

In [9]:
def traeSMInteracciones(fecha_max,fecha_tope):
    total_docs = 10000
    response = es.search(
        index= parametros.sm_interaciones,
        body={
            "_source": ["clr_txt_idbneficmtc","variable1","variable2","variable3"
                        ,"variable4","current_phase","clr_txt_assignment","owner_name"
                        ,"open_time","close_time","resolution","open","incident_id","source"]
            ,"query": {
              "range": {
                "open_time": {
                    "gte": fecha_max,
                    "lt": fecha_tope
                }
              }
          }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_SM_interactions = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    
    if datos_SM_interactions is None or datos_SM_interactions.empty:
        return pd.DataFrame()
    datos_SM_interactions = datos_SM_interactions.rename(columns={'variable1' : 'gestion.tablero14categoria'
                                                                  ,'variable2' : 'gestion.tablero14subcategoria'
                                                                  ,'current_phase':'gestion.tablero14estadoTicket'
                                                                  ,'owner_name':'gestion.tablero14usuarioTicket'
                                                                  ,'clr_txt_assignment':'gestion.tablero14responsable'
                                                                  ,'source':'gestion.tablero14canal'
                                                                  ,'clr_txt_idbneficmtc':'gestion.tablero14id_Beneficiario'
                                                                  ,'variable4' : 'gestion.tablero14Tipo4'
                                                                  ,'incident_id':'gestion.tablero14numeroTicket'})         
    datos_SM_interactions = datos_SM_interactions.dropna(subset=['gestion.tablero14id_Beneficiario'])
    datos_SM_interactions = datos_SM_interactions.drop(datos_SM_interactions[datos_SM_interactions['gestion.tablero14id_Beneficiario']=="null"].index)
    datos_SM_interactions['gestion.tablero14id_Beneficiario'] = datos_SM_interactions['gestion.tablero14id_Beneficiario'].astype(int)
    datos_SM_interactions['gestion.tablero14canal'] = datos_SM_interactions['gestion.tablero14canal'].replace(['5','6'],['Correo Electrónico','Teléfono'])

    return datos_SM_interactions

Realizando bucle hasta conseguir datos de servicemanager-interacciones o hasta la fecha actual para realizar la carga de datos 

In [10]:
fecha_max_mintic = fecha_ejecucion



fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
datos_SM_interactions = traeSMInteracciones(fecha_max_mintic,fecha_tope_mintic)


if datos_SM_interactions is None or datos_SM_interactions.empty:
    while (datos_SM_interactions is None or datos_SM_interactions.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        datos_SM_interactions = traeSMInteracciones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass



In [11]:
#fecha_max_mintic
#print (fecha_max_mintic, "   hastaaaa  ", fecha_tope_mintic)

Haciendo merge entre semilla e interacciones

Realizando inserción

In [12]:
use_these_keys = [
                  'gestion.tablero14location'
                  , 'gestion.tablero14id_Beneficiario'
                  , 'gestion.tablero14Tipo4'
                  , 'gestion.tablero14fechaApertura'
                  , 'gestion.tablero14nombreSede'
                  , 'gestion.tablero14dptoGestion'
                  , 'gestion.tablero14muniGestion'
                  , 'gestion.tablero14anyo'
                  , 'gestion.tablero14mes'
                  , 'gestion.tablero14dia'
                  , 'gestion.tablero14nombre_centro_pob'
                  , '@timestamp'
                  ]
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{str(document['site_id']) + '-' + str(document['gestion.tablero14numeroTicket']) + '-' + str(document['gestion.tablero14fechaApertura'])}",
                "_source": filterKeys(document),
            }

In [13]:
def doc_ordenamiento(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": 1,
                "_source": filterKeys(document),
            }

In [14]:
d = {'ordenar.tablero14ordenamiento': [1]}
df_ordenamiento = pd.DataFrame(data=d)
use_these_keys = [
'ordenar.tablero14ordenamiento'
]
salida = helpers.bulk(es, doc_ordenamiento(df_ordenamiento ))

In [15]:
datos_semilla.columns=['nombreSede', 'DDA', 'estadoInstalacion', 'COD_ISO', 'energia',
'nombre_departamento', 'codCentroPoblado', 'nombreInstitucionEd',
'codDanesede', 'tipoSitio', 'codDaneMuni', 'nombre_centro_pob',
'site_id', 'matricula', 'energiadesc', 'grupoDesc', 'cod_servicio',
'region', 'detalleSitio', 'nombre_municipio', 'gestion.tablero14id_Beneficiario',
'codDaneInstitucionEdu', 'gestion.tablero14location']

In [16]:
try:
    concat = pd.merge(datos_SM_interactions,datos_semilla, on=['gestion.tablero14id_Beneficiario'],how='inner')
    concat = concat.rename(columns={'id_Beneficiario' : 'gestion.tablero14id_Beneficiario'
                                    ,'variable4':'gestion.tablero14Tipo4'
                                    ,'variable3':'gestion.tablero14.Tipo3'
                                    ,'open_time':'gestion.tablero14fechaApertura'
                                    ,'nombreSede':'gestion.tablero14nombreSede'
                                    ,'nombre_departamento':'gestion.tablero14DepartamentoGestion'
                                    ,'nombre_municipio':'gestion.tablero14MunicipioGestion'
                                    ,'nombre_centro_pob':'gestion.tablero14nombre_centro_pob'
                                    })
    concat['gestion.tablero14fechaApertura'] = concat['gestion.tablero14fechaApertura'].str.replace("T"," ")
    concat['gestion.tablero14fechaApertura'] = concat['gestion.tablero14fechaApertura'].str.slice(stop=19)
    concat["gestion.tablero14anyo"] = concat["gestion.tablero14fechaApertura"].str[0:4]
    concat["gestion.tablero14mes"] = concat["gestion.tablero14fechaApertura"].str[5:7]
    concat["gestion.tablero14dia"] = concat["gestion.tablero14fechaApertura"].str[8:10]
    concat['@timestamp'] = now.isoformat()
    concat = concat.fillna('null')
     
    use_these_keys =   [
         'gestion.tablero14fechaApertura', 
         'gestion.tablero14Tipo4',
         'gestion.tablero14DepartamentoGestion',
         'gestion.tablero14MunicipioGestion',
         'gestion.tablero14id_Beneficiario', 
         'gestion.tablero14nombreSede', 
         'gestion.tablero14location', 
         'gestion.tablero14anyo',
         'gestion.tablero14mes', 
         'gestion.tablero14dia', 
         '@timestamp']
        
    salida = helpers.bulk(es, doc_generator(concat))
    print("Fecha: ", now,"- Interacciones insertadas en indice principal:",salida[0])
except Exception as e:
    print(e, "\nNinguna Interacción insertada en indice principal:")

    

Fecha:  2021-07-23 18:48:41.789675 - Interacciones insertadas en indice principal: 14


Actualizando fecha de control de ejecución

In [17]:
#concat

In [18]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion = '2021-04-29T00:00:00'

if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero14',
        body = { 'jerarquia-tablero14': 'jerarquia-tablero14','tablero14.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)


actualizada fecha control de ejecucion: 2021-07-01T00:10:00
